In [1]:
import random
import numpy as np
import bs4 as BS
import requests as rqs
import pandas as pd 

In [2]:
pages = random.sample(range(1, 193),192)
pages.sort()

In [3]:
year = []
car = []
price = []
for page in pages:

    source = rqs.get(r'https://www.cheki.co.ke/vehicles/toyota?page={}'.format(page)).text

    soup = BS.BeautifulSoup(source, 'lxml')

    for article in soup.find_all('div',class_= 'right-card-part-container'):

        year.append(article.find('span', class_='ellipses').text.split('\n')[2])

        car.append(article.find('span', class_='ellipses').text.split('\n')[3])
        try:

            price.append(article.find('h2', class_= 'listing-price').text)
        except AttributeError:
            price.append('0')

In [4]:
Dict = {
    'Car': car,
    'Year': year,
    'Price': price
}

In [5]:
df = pd.DataFrame(Dict)

In [6]:
df.head(2)

,Car,Year,Price
0,Toyota Crown DeLuxe 2300,2013,"\nKSh2,499,999\n"
1,Toyota Spade,2013,"\nKSh750,000\n"


In [7]:
df['Price'] = pd.to_numeric(df['Price'].apply(lambda x: x.replace('\n', '').replace('KSh', '').replace(',','')))

In [8]:
df.head(2)

,Car,Year,Price
0,Toyota Crown DeLuxe 2300,2013,2499999
1,Toyota Spade,2013,750000


In [9]:
filt = (df['Year'] == 'Toyota Vanguard')

In [10]:
incorrect = df[filt].index

In [11]:
df = df.drop(incorrect).reset_index(drop = True)

In [12]:
df['Year'] = pd.to_numeric(df['Year'])

In [13]:
df = df.rename(columns = {'Price': 'Price in KSh'})

In [14]:
df.head(2)

,Car,Year,Price in KSh
0,Toyota Crown DeLuxe 2300,2013,2499999
1,Toyota Spade,2013,750000


In [15]:
car = df.groupby('Car')

In [16]:
car.get_group('Toyota Spade').head(2)

,Car,Year,Price in KSh
1,Toyota Spade,2013,750000
23,Toyota Spade,2013,750000


In [17]:
pd.set_option('display.max_rows',1000)

In [18]:
df[df['Car'].str.contains('HiAce')].sort_values(by = 'Price in KSh').reset_index(drop = True).head(2)

,Car,Year,Price in KSh
0,Toyota HiAce,1988,465000
1,Toyota HiAce,1999,635000
